# Gold Layer – Star Schema for OULAD (Open University Learning Analytics Dataset)

This notebook builds the **Gold layer** of our Lakehouse for the **OULAD dataset**, using a
**star schema** design.

We assume the **Bronze (raw)** and **Silver (clean)** layers are already created in the catalog:

- Catalog: `analytics_oulad`
- Schemas:
  - `bronze_raw`     – raw CSV ingested as Delta
  - `silver_clean`   – cleaned, typed base tables
  - `gold_star`      – **(this notebook)** BI-ready star schema

The Gold layer will expose **dimension tables** and **fact tables** that can be consumed by
BI tools (Looker Studio, Power BI, etc.) and by downstream ML pipelines.


## 1. Star Schema Concepts

A **star schema** is a dimensional modeling pattern used in data warehouses and BI systems.
It consists of:

- **Fact tables**
  - Contain numeric, aggregatable measures (e.g., `score`, `clicks`)
  - Have foreign keys that reference dimensions
  - Represent business events at a specific **grain** (level of detail)

- **Dimension tables**
  - Contain descriptive attributes (e.g., `gender`, `region`, `activity_type`)
  - Provide context for slicing and dicing facts
  - Typically smaller and denormalized for fast querying

The structure looks like a ⭐ where the **fact** table is at the center and
the **dimensions** are the points of the star.

In this project we will build:

**Dimensions**
- `gold_star.dim_student`
- `gold_star.dim_course`
- `gold_star.dim_assessment`

**Facts**
- `gold_star.fact_assessment_score`
- `gold_star.fact_vle_interactions`


In [0]:
%sql
USE CATALOG analytics_oulad;
USE SCHEMA gold_star;

## 2. Grain (Level of Detail) of Fact Tables

Before creating tables, we must define the **grain** for each fact table.

### 2.1 `fact_assessment_score`
Each row represents **one student's result for one assessment**.

Grain:
> *"Student × Assessment"*

Key columns:
- `student_id`
- `assessment_id`

Measures:
- `score`
- `is_banked` (banked result)
- `date_submitted` (for timeliness analysis)

Context columns:
- `code_module`
- `code_presentation`
- `assessment_type`
- `weight`

---

### 2.2 `fact_vle_interactions`
Each row represents **one student's interactions with one VLE site on one day**.

Grain:
> *"Student × VLE Site × Date"*

Key columns:
- `student_id`
- `id_site`
- `date`

Measures:
- `clicks` (number of clicks)

Context columns:
- `code_module`
- `code_presentation`
- `activity_type`


## 3. Dimension Tables

### 3.1 `dim_student`
Describes **who** the student is.

Columns (attributes):
- `student_id` (natural key from OULAD)
- `gender`
- `region`
- `highest_education`
- `imd_band` (deprivation index band)
- `age_band`
- `disability`

### 3.2 `dim_course`
Describes **which course and presentation** the data relates to.

Columns:
- `code_module`
- `code_presentation`
- `presentation_length` (days)

### 3.3 `dim_assessment`
Describes **what assessment** the score belongs to.

Columns:
- `assessment_id`
- `code_module`
- `code_presentation`
- `assessment_type` (TMA, CMA, exam, etc.)
- `assessment_date` (day number relative to course start)
- `weight` (contribution to final score)

> **Note:** We are using **natural keys** (e.g., `student_id`, `assessment_id`) as primary keys here.
> In a larger production system we might introduce **surrogate keys** (integer identities)
> to decouple the warehouse from source system changes.


In [0]:
%sql
CREATE OR REPLACE TABLE analytics_oulad.gold_star.dim_student AS
SELECT DISTINCT
    student_id,
    gender,
    region,
    highest_education,
    imd_band,
    disability
FROM analytics_oulad.silver_clean.student_info_base;


In [0]:
%sql
SELECT * FROM analytics_oulad.gold_star.dim_student
LIMIT 5

In [0]:
%sql
CREATE OR REPLACE TABLE analytics_oulad.gold_star.dim_course AS
SELECT DISTINCT
    code_module,
    code_presentation,
    presentation_length
FROM analytics_oulad.silver_clean.courses_base;

In [0]:
%sql
CREATE OR REPLACE TABLE analytics_oulad.gold_star.dim_assessment AS
SELECT
    assessment_id,
    code_module,
    code_presentation,
    assessment_type,
    assessment_date,
    weight
FROM analytics_oulad.silver_clean.assessments_base;


## 4. Fact Tables

Fact tables store the **measures** we want to analyze.

### 4.1 `fact_assessment_score`

Business question examples:
- What is the **average score** by module?
- How does performance vary by **gender** or **region**?
- How many **late submissions** occur per assessment?

Columns:
- `student_id` (FK → `dim_student.student_id`)
- `assessment_id` (FK → `dim_assessment.assessment_id`)
- `date_submitted`
- `score`
- `is_banked`
- `code_module`, `code_presentation`, `assessment_type`, `weight` (denormalized for convenience)

Measures:
- `score` (0–100)
- Derived metrics (e.g., pass/fail, late flag) can be added later.

---

### 4.2 `fact_vle_interactions`

Business question examples:
- Does **VLE engagement** predict student success?
- How many clicks per week per module?
- Which content types (`activity_type`) drive the most engagement?

Columns:
- `student_id` (FK → `dim_student.student_id`)
- `id_site` (FK → `dim_course` indirectly through `code_module`, `code_presentation`)
- `date`
- `clicks`
- `code_module`, `code_presentation`, `activity_type`

Measures:
- `clicks` (number of interactions)


In [0]:
%sql
CREATE OR REPLACE TABLE analytics_oulad.gold_star.fact_assessment_score AS
SELECT
    sa.student_id,
    sa.assessment_id,
    sa.date_submitted,
    sa.score,
    sa.is_banked,
    a.code_module,
    a.code_presentation,
    a.assessment_type,
    a.weight
FROM analytics_oulad.silver_clean.student_assessment_base sa
LEFT JOIN analytics_oulad.silver_clean.assessments_base a
    ON sa.assessment_id = a.assessment_id;

In [0]:
%sql
ALTER TABLE analytics_oulad.gold_star.fact_assessment_score
ADD COLUMN is_pass BOOLEAN;

UPDATE analytics_oulad.gold_star.fact_assessment_score
SET is_pass = CASE WHEN score >= 40 THEN TRUE ELSE FALSE END;

In [0]:
%sql
CREATE OR REPLACE TABLE analytics_oulad.gold_star.fact_vle_interactions AS
SELECT
    sv.student_id,
    sv.id_site,
    sv.date,
    sv.clicks,
    v.code_module,
    v.code_presentation,
    v.activity_type
FROM analytics_oulad.silver_clean.student_vle_base sv
LEFT JOIN analytics_oulad.silver_clean.vle_base v
    ON sv.id_site = v.id_site;

## 5. Star Schema Diagram (Text-based)

Below is a logical view of the star schema we just created.

### 5.1 Assessment Performance Star

                 dim_course
                (module/presentation)
                     ▲
                     │
           dim_assessment       dim_student
           (what is tested)     (who)
                     ▲           ▲
                     │           │
                     └──── fact_assessment_score ────▶ measures:
                             (student × assessment)   - score
                                                     - is_banked
                                                     - is_pass (derived)

### 5.2 VLE Engagement Star

               dim_course
            (module/presentation)
                     ▲
                     │
             dim_student
                (who)
                     ▲
                     │
                     └──── fact_vle_interactions ───▶ measures:
                             (student × site × date) - clicks


## 6. Quality Checks & Example Queries

We perform basic sanity checks to validate the Gold layer:

1. Row counts
2. Joinability between facts and dimensions
3. Simple aggregations

These checks also serve as usage examples for BI and analytics.

In [0]:
%sql
SELECT 'dim_student' AS table_name, COUNT(*) AS row_count
FROM analytics_oulad.gold_star.dim_student
UNION ALL
SELECT 'dim_course', COUNT(*) FROM analytics_oulad.gold_star.dim_course
UNION ALL
SELECT 'dim_assessment', COUNT(*) FROM analytics_oulad.gold_star.dim_assessment
UNION ALL
SELECT 'fact_assessment_score', COUNT(*) FROM analytics_oulad.gold_star.fact_assessment_score
UNION ALL
SELECT 'fact_vle_interactions', COUNT(*) FROM analytics_oulad.gold_star.fact_vle_interactions;

In [0]:
%sql
--Average score by module
SELECT
    code_module,
    AVG(score) AS avg_score
FROM analytics_oulad.gold_star.fact_assessment_score
GROUP BY code_module
ORDER BY avg_score DESC;

In [0]:
%sql
-- Clicks vs. student
SELECT
    student_id,
    SUM(clicks) AS total_clicks
FROM analytics_oulad.gold_star.fact_vle_interactions
GROUP BY student_id
ORDER BY total_clicks DESC
LIMIT 20;


## 7. Summary

In this notebook we:

1. **Defined the Gold layer design** using dimensional modeling concepts:
   - Star schema
   - Fact tables and dimension tables
   - Grain and keys

2. **Created dimension tables**:
   - `gold_star.dim_student`
   - `gold_star.dim_course`
   - `gold_star.dim_assessment`

3. **Created fact tables**:
   - `gold_star.fact_assessment_score` (student × assessment)
   - `gold_star.fact_vle_interactions` (student × site × date)

4. **Validated** the model with basic row counts and example queries.

The Gold layer is now **ready to be consumed** by:
- BI tools (Looker Studio, Power BI, Tableau)
- Machine Learning notebooks (e.g., student dropout prediction)
- Databricks SQL dashboards.

Next steps could include:
- Adding **surrogate keys** and slowly changing dimensions (SCDs)
- Creating **materialized views** for common dashboards
- Building **ML features** directly from the Gold layer.
